In [1]:
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns

import os

In [3]:
""" Data locations
"""

unpd_root = "../../data/unpd_data"
gbd_root = "../../data/gbd_data"



# UN Data

Extract a "World" subset for working with on plotting, global forecasts and working out next 8BN

In [12]:
unpd_scenarios = ["High","Medium","Low"]

In [4]:
df_unpd_medium_indicators = pd.read_csv(
    os.path.join(unpd_root, "WPP2022_Demographic_Indicators_Medium.csv"),
    low_memory=False
)
df_unpd_other_indicators = pd.read_csv(
    os.path.join(unpd_root, "WPP2022_Demographic_Indicators_OtherVariants.csv"),
    low_memory=False
)

In [5]:
df_unpd_all_indicators = pd.concat(
    [df_unpd_medium_indicators, df_unpd_other_indicators]
)

In [13]:
df_unpd_all_indicators_scenario_filter = df_unpd_all_indicators[
    df_unpd_all_indicators.Variant.apply(lambda v: v in unpd_scenarios)
]

In [18]:
df_unpd_world_indicators = df_unpd_all_indicators[df_unpd_all_indicators.Location=="World"]
                            [df_unpd_all_indicators.Variant.apply(lambda v: v in unpd_scenarios)]

/tmp/ipykernel_48466/2012889876.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_unpd_world_indicators = (df_unpd_all_indicators[df_unpd_all_indicators.Location=="World"])[df_unpd_all_indicators.Variant.apply(lambda v: v in unpd_scenarios)]


IndexError: indices are out-of-bounds

In [8]:
df_unpd_world_indicators

,SortOrder,LocID,Notes,ISO3_code,ISO2_code,SDMX_code,LocTypeID,LocTypeName,ParentID,Location,...,Q0060Male,Q0060Female,Q1550,Q1550Male,Q1550Female,Q1560,Q1560Male,Q1560Female,NetMigrations,CNMR
0,1,900,NaN,NaN,NaN,1.0,1,World,0,World,...,580.7496,498.0396,240.3164,271.6253,208.1916,378.6968,430.2594,324.9308,0.0,0.0
1,1,900,NaN,NaN,NaN,1.0,1,World,0,World,...,566.7283,490.1993,231.1772,258.0905,203.7797,368.3186,415.8362,319.3364,0.0,0.0
2,1,900,NaN,NaN,NaN,1.0,1,World,0,World,...,546.3173,477.2639,218.6745,240.0344,197.1422,353.0546,395.5327,309.9103,0.0,0.0
3,1,900,NaN,NaN,NaN,1.0,1,World,0,World,...,535.8289,469.5325,212.8720,232.6019,193.0492,345.0826,385.8433,303.9053,0.0,0.0
4,1,900,NaN,NaN,NaN,1.0,1,World,0,World,...,523.1236,458.4842,205.7622,224.0498,187.4439,335.4420,374.6576,295.9944,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,1,900,NaN,NaN,NaN,1.0,1,World,0,World,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1116,1,900,NaN,NaN,NaN,1.0,1,World,0,World,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117,1,900,NaN,NaN,NaN,1.0,1,World,0,World,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1118,1,900,NaN,NaN,NaN,1.0,1,World,0,World,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_unpd_world_indicators.columns

Index(['SortOrder', 'LocID', 'Notes', 'ISO3_code', 'ISO2_code', 'SDMX_code',
       'LocTypeID', 'LocTypeName', 'ParentID', 'Location', 'VarID', 'Variant',
       'Time', 'TPopulation1Jan', 'TPopulation1July', 'TPopulationMale1July',
       'TPopulationFemale1July', 'PopDensity', 'PopSexRatio', 'MedianAgePop',
       'NatChange', 'NatChangeRT', 'PopChange', 'PopGrowthRate',
       'DoublingTime', 'Births', 'Births1519', 'CBR', 'TFR', 'NRR', 'MAC',
       'SRB', 'Deaths', 'DeathsMale', 'DeathsFemale', 'CDR', 'LEx', 'LExMale',
       'LExFemale', 'LE15', 'LE15Male', 'LE15Female', 'LE65', 'LE65Male',
       'LE65Female', 'LE80', 'LE80Male', 'LE80Female', 'InfantDeaths', 'IMR',
       'LBsurvivingAge1', 'Under5Deaths', 'Q5', 'Q0040', 'Q0040Male',
       'Q0040Female', 'Q0060', 'Q0060Male', 'Q0060Female', 'Q1550',
       'Q1550Male', 'Q1550Female', 'Q1560', 'Q1560Male', 'Q1560Female',
       'NetMigrations', 'CNMR'],
      dtype='object')

In [10]:
df_unpd_world_indicators.to_csv("../../data/prepared_data/unpd_combined_indicators.csv")

In [11]:
df_unpd_world_indicators.to_csv("../../data/prepared_data/unpd_world_indicators.csv") # for global stats and working out 8BN

# Global Burden of Disease

Do the same thing with the GBD study